In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from scipy.stats import norm
import seaborn as sns
from geopy import Nominatim
import base64
from IPython.html.widgets import interact #Needed for interactive graph
#from pygeocoder import geocoder
import folium
import json
mappingDict={'Zürich': 'Zürich', 'Bern': 'Bern', 'Luzern': 'Luzern', 'Uri': 'Uri', 'Schwyz': 'Schwyz', 'Obwalden': 'Obwalden', 'Nidwalden': 'Nidwalden', 'Glarus': 'Glarus', 'Zug': 'Zug', 'Freiburg': 'Fribourg/Freiburg', 'Solothurn': 'Solothurn', 'Basel-Stadt': 'Basel-Stadt', 'Basel-Landschaft': 'Basel-Landschaft', 'Schaffhausen': 'Schaffhausen', 'Appenzell Ausserrhoden': 'Appenzel', 'Appenzell Innerrhoden': 'Appenzel', 'St. Gallen': 'St', 'Graubünden': 'Graubünden/Grigioni/Grischun', 'Aargau': 'Aargau', 'Thurgau': 'Thurgau', 'Tessin': 'Ticino', 'Waadt': 'Vaud', 'Wallis': 'Valais/Wallis', 'Neuenburg': 'Neuchâtel', 'Genf': 'Genève', 'Jura': 'Jura'}
#Replace some manually
mappingDict['Appenzell Ausserrhoden']='Appenzell Ausserrhoden'
mappingDict['Appenzell Innerrhoden']='Appenzell Innerrhoden'
mappingDict['Freiburg']= 'Fribourg'
mappingDict['Graubünden']='Graubünden/Grigioni'
mappingDict['St. Gallen']='St. Gallen'
mappingDict['Bern']='Bern/Berne'
beginLocation=[46.911492, 8.090388]
beginZoom=7
dfCantonAge=pd.read_excel('rateSuisseCantonAgeGerman.xlsx')
dfCantonAge.columns.values[1] = 'Agegroup'
dfCantonAge=dfCantonAge[['Kanton','Agegroup','Gesamt']].iloc[1:-1]
for index,agegroup in enumerate(dfCantonAge["Agegroup"]):
    #Remove the ans
    cleaned=str(agegroup).split()[0]
    if cleaned=="60":
        cleaned=cleaned+"+"
    dfCantonAge.iloc[index, dfCantonAge.columns.get_loc('Agegroup')]=cleaned
#In the datarame the unkown values are replaced with ... 
#We replace the ... with NaN
dfCantonAge.replace("...",np.nan,inplace=True)
dfCantonAge["Gesamt"]=dfCantonAge["Gesamt"].astype(float)
dfCantonAge.columns=["Canton","Agegroup","Unemployement"]
for index,name in enumerate(dfCantonAge["Canton"]):
    dfCantonAge.iloc[index, dfCantonAge.columns.get_loc('Canton')] = mappingDict[name]

#The goal is to get a dictionairy {district{agegroup{color{}}}}
colorDict={}
colors=sns.color_palette("GnBu",5)
agegroups=[]
percentageCutDict={}
ageGroupDfDict={}
for agegroup,ageGroupDf in dfCantonAge.groupby("Agegroup"):
    agegroups.append(agegroup)
    ageGroupDfDict[agegroup]=ageGroupDf
    #Find the cuting percentages of this agegroup.
    percentageCuts=pd.qcut(ageGroupDf["Unemployement"],5, labels=False,retbins=True, precision=1, duplicates='raise')[-1]
    percentageCutDict[agegroup]=percentageCuts
    for canton,cantonageDf in ageGroupDf.groupby("Canton"):
        value=cantonageDf["Unemployement"].values[0]
        #Categorise this value in colors using the cutting percentages.
        classified=False
        for index,percentage in enumerate(percentageCuts):
            if index<len(percentageCuts)-1:
                if percentageCuts[index]<=value:
                    if value<percentageCuts[index+1]:
                        color=colors[index]
                        if canton in colorDict:
                            colorDict[canton][agegroup]=color
                        else:
                            colorDict[canton]={agegroup:color}
                        classified=True
            if classified==True:
                break
        if classified==False:
            if not np.isnan(value):
                assert(value==percentageCuts[-1])
                color=colors[-1]
                if canton in colorDict:
                    colorDict[canton][agegroup]=color
                else:
                    colorDict[canton]={agegroup:color}


/Users/adriaan/anaconda/lib/python3.6/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [ ]:
import os
if not os.path.exists("figures"):
    os.makedirs("figures")
for canton,ageDf in dfCantonAge.groupby("Canton"):
    plt.close()
    sns.set_style("whitegrid")
    plt.xlabel("age group")
    plt.ylabel("unemployement")
    colorPallete=[colorDict[canton][agegroup] for agegroup in agegroups if agegroup in colorDict[canton]]
    ax=sns.barplot(x="Agegroup", y="Unemployement", data=ageDf.reset_index(),palette=colorPallete)
    ax.set(xlabel='age groups', ylabel='percentage',title="Unemployement rate vs age in: "+canton)
    #Save the barplots for later (remove the / because of directory problems)
    if "/" in canton:
        canton=canton.replace("/","_")
    plt.savefig("figures/"+canton+".png")

locations={}
geolocator=Nominatim()
for canton,ageDf in dfCantonAge.groupby("Canton"):
    location = geolocator.geocode("Schweiz " + canton )
    if location==None:
        location=geolocator.geocode("Switzerland"+canton)
    locations[canton]=location
    
def create_map (age):
    """
    Function to create the map for a specific agegroup.
    """
    resolution, width, height = 75, 5, 2
    colorscheme=percentageCutDict[age]
    s = folium.Map(location=[46.941500, 8.398005], zoom_start=7.5)
    topo_json_data = json.load(open('ch-cantons.topojson.json'))
    s.choropleth(
    geo_data=topo_json_data,
    name='choropleth',
    data=ageGroupDfDict[age],
    columns=['Canton', 'Unemployement'],
    key_on='feature.properties.name',
    threshold_scale = list(colorscheme),
    fill_color= 'GnBu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Overall unemployement rate for age group: '+str(age),
    topojson ='objects.cantons'
    )
    
    for canton,ageDf in dfCantonAge.groupby("Canton"):
        #Insert the image in a html iframe.
        html = '<img src="data:image/png;base64,{}" style="width: 100%; height: 100%" />'.format
        cantonName=canton
        if "/" in canton:
            cantonName=cantonName.replace("/","_")
            
        encoded =  base64.b64encode(open("figures/"+cantonName+".png", 'rb').read()).decode()


        iframe = folium.IFrame(html(encoded), width=(width*resolution)+20, height=(height*resolution)+20)
        popup = folium.Popup(iframe, max_width=2650)
        icon = folium.Icon(color="blue",icon_color="white",icon="info-sign")
        marker = folium.Marker(location=[locations[canton].latitude, locations[canton].longitude], popup=popup, icon=icon)
        s.add_child(marker);
    return s


In [ ]:
s=interact(create_map,age=agegroups)
s